In [1]:
from utils.my_llm_utils import *
from utils.my_utils import *
from utils.tavily_search_utils import web_search

In [2]:
csv_path = "data/AI Partnerships sample_updated.csv"

# read the csv file
df, output_path = read_csv_with_output_path(csv_path)


In [ ]:
# # load the LLM

# # for default llm
# #llm = get_llm()

# # for openai llm
# #llm = get_llm(use_openai=True, model_name="gpt-4o")

# # for ollama llm - [gemma3:1b, llama3.2:latest, deepseek-r1:8b, phi4:latest]
# llm = get_llm(use_openai=False, model_name="gemma3:1b")


# # Check which llm has been loaded
# if hasattr(llm, 'model'):
# 	print(llm.model)  # check loaded model
# else:
# 	print(llm)

gemma3:1b


In [3]:
import wandb
wandb.init(project="AI-Partnerships-agent")

wandb: Currently logged in as: mikhailry (mikhailry-paylocity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
ollamaModels = ["gemma3:1b", "llama3.2:latest", "phi4:latest"]
openAiModels = ["gpt-4o"]

prompt = f"""
    Summarize the following AI partnership announcement in one sentence:
    """

In [5]:
import time
import numpy as np
from datetime import datetime

# Initialize statistics dictionary
model_stats = {}

def process_model_summaries(models, use_openai=False):
    for model in models:
        print(f"\nProcessing {model}...")
        llm = get_llm(use_openai=use_openai, model_name=model)
        
        # Initialize statistics for this model
        model_stats[model] = {
            'summary_lengths': [],
            'processing_times': [],
            'start_time': datetime.now()
        }
        
        # iterate over the rows of the dataframe
        for index, row in df.iterrows():
            # get the raw content
            raw_content = row['raw_content']
            
            # Time the summarization
            start_time = time.time()
            # summarize the raw content
            summary = llm.invoke(prompt + "\n" + raw_content)
            end_time = time.time()
            
            # Store statistics
            model_stats[model]['summary_lengths'].append(len(str(summary)))
            model_stats[model]['processing_times'].append(end_time - start_time)
            
            # add the summary to the dataframe
            df.at[index, model + '_summary'] = summary
            
        # Calculate final statistics
        model_stats[model]['end_time'] = datetime.now()
        model_stats[model]['total_time'] = (model_stats[model]['end_time'] - model_stats[model]['start_time']).total_seconds()
        model_stats[model]['avg_time'] = np.mean(model_stats[model]['processing_times'])
        model_stats[model]['min_time'] = np.min(model_stats[model]['processing_times'])
        model_stats[model]['max_time'] = np.max(model_stats[model]['processing_times'])
        model_stats[model]['median_time'] = np.median(model_stats[model]['processing_times'])
        model_stats[model]['avg_length'] = np.mean(model_stats[model]['summary_lengths'])
        model_stats[model]['min_length'] = np.min(model_stats[model]['summary_lengths'])
        model_stats[model]['max_length'] = np.max(model_stats[model]['summary_lengths'])
        model_stats[model]['median_length'] = np.median(model_stats[model]['summary_lengths'])

# Process Ollama models
process_model_summaries(ollamaModels, use_openai=False)

# Process OpenAI models
process_model_summaries(openAiModels, use_openai=True)

# Create summary DataFrame
summary_stats = pd.DataFrame({
    'Model': list(model_stats.keys()),
    'Total Time (s)': [stats['total_time'] for stats in model_stats.values()],
    'Avg Time (s)': [stats['avg_time'] for stats in model_stats.values()],
    'Min Time (s)': [stats['min_time'] for stats in model_stats.values()],
    'Max Time (s)': [stats['max_time'] for stats in model_stats.values()],
    'Median Time (s)': [stats['median_time'] for stats in model_stats.values()],
    'Avg Length': [stats['avg_length'] for stats in model_stats.values()],
    'Min Length': [stats['min_length'] for stats in model_stats.values()],
    'Max Length': [stats['max_length'] for stats in model_stats.values()],
    'Median Length': [stats['median_length'] for stats in model_stats.values()]
})

# Log both tables to wandb
wandb.log({
    "summaries": wandb.Table(dataframe=modelComparison),
    "model_stats": wandb.Table(dataframe=summary_stats)
})

# Display the summary statistics
print("\nModel Performance Statistics:")
print(summary_stats.to_string(index=False))


Processing gemma3:1b...

Processing llama3.2:latest...

Processing phi4:latest...

Processing gpt-4o...


TypeError: object of type 'AIMessage' has no len()

In [ ]:
# for model in ollamaModels:
#     llm = get_llm(use_openai=False, model_name=model)
    
#     # iterate over the rows of the dataframe
#     for index, row in df.iterrows():
#         # get the raw content
#         raw_content = row['raw_content']
#         # summarize the raw content
#         summary = llm.invoke(prompt + "\n" + raw_content)
#         # add the summary to the dataframe
#         df.at[index, model + '_summary'] = summary

# for model in openAiModels:
#     llm = get_llm(use_openai=True)
#     # iterate over the rows of the dataframe
#     for index, row in df.iterrows():
#         # get the raw content
#         raw_content = row['raw_content']
#         # summarize the raw content
#         summary = llm.invoke(prompt + "\n" + raw_content)
#         # add the summary to the dataframe
#         df.at[index, model + '_summary'] = summary


In [13]:
# # select df columns: partner1, partner2, raw_content, gemma3:1b_summary, llama3.2:latest_summary, phi4:latest_summary, gpt-4o_summary and save as modelComparison data frame
# modelComparison = df[['partner1', 'partner2', 'raw_content', 'gemma3:1b_summary', 'llama3.2:latest_summary', 'phi4:latest_summary', 'gpt-4o_summary']].copy()

# # Convert AIMessage objects to strings
# for col in modelComparison.columns:
#     if col.endswith('_summary'):
#         modelComparison[col] = modelComparison[col].apply(lambda x: x.content if hasattr(x, 'content') else str(x))

# # Log to wandb
# wandb.log({"summaries": wandb.Table(dataframe=modelComparison)})